# **Laboratorio 8: Ready, Set, Deploy! 👩‍🚀👨‍🚀**

<center><strong>MDS7202: Laboratorio de Programación Científica para Ciencia de Datos - Primavera 2025</strong></center>

### Cuerpo Docente:

- Profesores: Diego Cortez, Gabriel Iturra
- Auxiliares: Melanie Peña, Valentina Rojas
- Ayudantes: Nicolás Cabello, Cristopher Urbina

### Equipo: SUPER IMPORTANTE - notebooks sin nombre no serán revisados

- Nombre de alumno 1:Andres Oñate
- Nombre de alumno 2:Javier Zapata

### **Link de repositorio de GitHub:** [Insertar Repositorio](https://github.com/andresignacio-o/MDS7202)

## Temas a tratar

- Entrenamiento y registro de modelos usando MLFlow.
- Despliegue de modelo usando FastAPI
- Containerización del proyecto usando Docker

## Reglas:

- **Grupos de 2 personas**
- Fecha de entrega: Entregas Martes a las 23:59.
- Instrucciones del lab el viernes a las 16:15 en formato online. Asistencia no es obligatoria, pero se recomienda **fuertemente** asistir.
- <u>Prohibidas las copias</u>. Cualquier intento de copia será debidamente penalizado con el reglamento de la escuela.
- Tienen que subir el laboratorio a u-cursos y a su repositorio de github. Labs que no estén en u-cursos no serán revisados. Recuerden que el repositorio también tiene nota.
- Cualquier duda fuera del horario de clases al foro. Mensajes al equipo docente serán respondidos por este medio.
- Pueden usar cualquier material del curso que estimen conveniente.

### Objetivos principales del laboratorio

- Generar una solución a un problema a partir de ML
- Desplegar su solución usando MLFlow, FastAPI y Docker

El laboratorio deberá ser desarrollado sin el uso indiscriminado de iteradores nativos de python (aka "for", "while"). La idea es que aprendan a exprimir al máximo las funciones optimizadas que nos entrega `pandas`, las cuales vale mencionar, son bastante más eficientes que los iteradores nativos sobre DataFrames.

# **Introducción**

<p align="center">
  <img src="https://media.giphy.com/media/v1.Y2lkPTc5MGI3NjExODJnMHJzNzlkNmQweXoyY3ltbnZ2ZDlxY2c0aW5jcHNzeDNtOXBsdCZlcD12MV9pbnRlcm5hbF9naWZfYnlfaWQmY3Q9Zw/AbPdhwsMgjMjax5reo/giphy.gif" width="400">
</p>



Consumida en la tristeza el despido de Renacín, Smapina ha decaído en su desempeño, lo que se ha traducido en un irregular tratamiento del agua. Esto ha implicado una baja en la calidad del agua, llegando a haber algunos puntos de la comuna en la que el vital elemento no es apto para el consumo humano. Es por esto que la sanitaria pública de la municipalidad de Maipú se ha contactado con ustedes para que le entreguen una urgente solución a este problema (a la vez que dejan a Smapina, al igual que Renacín, sin trabajo 😔).

El problema que la empresa le ha solicitado resolver es el de elaborar un sistema que les permita saber si el agua es potable o no. Para esto, la sanitaria les ha proveido una base de datos con la lectura de múltiples sensores IOT colocados en diversas cañerías, conductos y estanques. Estos sensores señalan nueve tipos de mediciones químicas y más una etiqueta elaborada en laboratorio que indica si el agua es potable o no el agua.

La idea final es que puedan, en el caso que el agua no sea potable, dar un aviso inmediato para corregir el problema. Tenga en cuenta que parte del equipo docente vive en Maipú y su intoxicación podría implicar graves problemas para el cierre del curso.

Atributos:

1. pH value
2. Hardness
3. Solids (Total dissolved solids - TDS)
4. Chloramines
5. Sulfate
6. Conductivity
7. Organic_carbon
8. Trihalomethanes
9. Turbidity

Variable a predecir:

10. Potability (1 si es potable, 0 no potable)

Descripción de cada atributo se pueden encontrar en el siguiente link: [dataset](https://www.kaggle.com/adityakadiwal/water-potability)

# **1. Optimización de modelos con Optuna + MLFlow (2.0 puntos)**

El objetivo de esta sección es que ustedes puedan combinar Optuna con MLFlow para poder realizar la optimización de los hiperparámetros de sus modelos.

Como aún no hemos hablado nada sobre `MLFlow` cabe preguntarse: **¡¿Qué !"#@ es `MLflow`?!**

<p align="center">
  <img src="https://media.tenor.com/eusgDKT4smQAAAAC/matthew-perry-chandler-bing.gif" width="400">
</p>

## **MLFlow**

`MLflow` es una plataforma de código abierto que simplifica la gestión y seguimiento de proyectos de aprendizaje automático. Con sus herramientas, los desarrolladores pueden organizar, rastrear y comparar experimentos, además de registrar modelos y controlar versiones.

<p align="center">
  <img src="https://spark.apache.org/images/mlflow-logo.png" width="350">
</p>

Si bien esta plataforma cuenta con un gran número de herramientas y funcionalidades, en este laboratorio trabajaremos con dos:
1. **Runs**: Registro que constituye la información guardada tras la ejecución de un entrenamiento. Cada `run` tiene su propio run_id, el cual sirve como identificador para el entrenamiento en sí mismo. Dentro de cada `run` podremos acceder a información como los hiperparámetros utilizados, las métricas obtenidas, las librerías requeridas y hasta nos permite descargar el modelo entrenado.
2. **Experiments**: Se utilizan para agrupar y organizar diferentes ejecuciones de modelos (`runs`). En ese sentido, un experimento puede agrupar 1 o más `runs`. De esta manera, es posible también registrar métricas, parámetros y archivos (artefactos) asociados a cada experimento.

### **Todo bien pero entonces, ¿cómo se usa en la práctica `MLflow`?**

Es sencillo! Considerando un problema de machine learning genérico, podemos registrar la información relevante del entrenamiento ejecutando `mlflow.autolog()` antes entrenar nuestro modelo. Veamos este bonito ejemplo facilitado por los mismos creadores de `MLflow`:

```python
#!pip install mlflow
import mlflow # importar mlflow

from sklearn.model_selection import train_test_split
from sklearn.datasets import load_diabetes
from sklearn.ensemble import RandomForestRegressor

db = load_diabetes()
X_train, X_test, y_train, y_test = train_test_split(db.data, db.target)

# Create and train models.
rf = RandomForestRegressor(n_estimators=100, max_depth=6, max_features=3)

mlflow.autolog() # registrar automáticamente información del entrenamiento
with mlflow.start_run(): # delimita inicio y fin del run
    # aquí comienza el run
    rf.fit(X_train, y_train) # train the model
    predictions = rf.predict(X_test) # Use the model to make predictions on the test dataset.
    # aquí termina el run
```

Si ustedes ejecutan el código anterior en sus máquinas locales (desde un jupyter notebook por ejemplo) se darán cuenta que en su directorio *root* se ha creado la carpeta `mlruns`. Esta carpeta lleva el tracking de todos los entrenamientos ejecutados desde el directorio root (importante: si se cambian de directorio y vuelven a ejecutar el código anterior, se creará otra carpeta y no tendrán acceso al entrenamiento anterior). Para visualizar estos entrenamientos, `MLflow` nos facilita hermosa interfaz visual a la que podemos acceder ejecutando:

```
mlflow ui
```

y luego pinchando en la ruta http://127.0.0.1:5000 que nos retorna la terminal. Veamos en vivo algunas de sus funcionalidades!

<p align="center">
  <img src="https://media4.giphy.com/media/v1.Y2lkPTc5MGI3NjExZXVuM3A5MW1heDFpa21qbGlwN2pyc2VoNnZsMmRzODZxdnluemo2bCZlcD12MV9pbnRlcm5hbF9naWZfYnlfaWQmY3Q9Zw/3o84sq21TxDH6PyYms/giphy.gif" width="400">
</p>

Les dejamos también algunos comandos útiles:

- `mlflow.create_experiment("nombre_experimento")`: Les permite crear un nuevo experimento para agrupar entrenamientos
- `mlflow.log_metric("nombre_métrica", métrica)`: Les permite registrar una métrica *custom* bajo el nombre de "nombre_métrica"


## **1.1 Combinando Optuna + MLflow (2.0 puntos)**

Ahora que tenemos conocimiento de ambas herramientas, intentemos ahora combinarlas para **más sabor**. El objetivo de este apartado es simple: automatizar la optimización de los parámetros de nuestros modelos usando `Optuna` y registrando de forma automática cada resultado en `MLFlow`.

Considerando el objetivo planteado, se le pide completar la función `optimize_model`, la cual debe:
- **Optimizar los hiperparámetros del modelo `XGBoost` usando `Optuna`.**
- **Registrar cada entrenamiento en un experimento nuevo**, asegurándose de que la métrica `f1-score` se registre como `"valid_f1"`. No se deben guardar todos los experimentos en *Default*; en su lugar, cada `experiment` y `run` deben tener nombres interpretables, reconocibles y diferentes a los nombres por defecto (por ejemplo, para un run: "XGBoost con lr 0.1").
- **Guardar los gráficos de Optuna** dentro de una carpeta de artefactos de Mlflow llamada `/plots`.
- **Devolver el mejor modelo** usando la función `get_best_model` y serializarlo en el disco con `pickle.dump`. Luego, guardar el modelo en la carpeta `/models`.
- **Guardar el código en `optimize.py`**. La ejecución de `python optimize.py` debería ejecutar la función `optimize_model`.
- **Guardar las versiones de las librerías utilizadas** en el desarrollo.
- **Respalde las configuraciones del modelo final y la importancia de las variables** en un gráfico dentro de la carpeta `/plots` creada anteriormente.

*Hint: Le puede ser útil revisar los parámetros que recibe `mlflow.start_run`*

```python
def get_best_model(experiment_id):
    runs = mlflow.search_runs(experiment_id)
    best_model_id = runs.sort_values("metrics.valid_f1")["run_id"].iloc[0]
    best_model = mlflow.sklearn.load_model("runs:/" + best_model_id + "/model")

    return best_model
```

Ver en optimize.py

In [5]:
import os, json, tempfile, pickle
from datetime import datetime

import numpy as np
import mlflow
import mlflow.sklearn
import optuna

from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split
from sklearn.datasets import make_classification

from xgboost import XGBClassifier
import matplotlib.pyplot as plt
from optuna.visualization.matplotlib import plot_optimization_history, plot_param_importances

In [ ]:
import xgboost as XGBClassifier # Ya lo tienes
from xgboost import DMatrix # Añadir esta
from xgboost.callback import EarlyStopping # Necesaria para callbacks

In [6]:
def get_best_model(experiment_id):
    runs = mlflow.search_runs(experiment_id)
    best_model_id = runs.sort_values("metrics.valid_f1")["run_id"].iloc[0]
    best_model = mlflow.sklearn.load_model("runs:/" + best_model_id + "/model")
    return best_model

In [7]:
def _log_versions():
    import mlflow as _mlf, optuna as _opt, xgboost as _xgb, sklearn as _sk, numpy as _np
    versions = {
        "python": f"{os.sys.version_info.major}.{os.sys.version_info.minor}.{os.sys.version_info.micro}",
        "mlflow": _mlf.__version__, "optuna": _opt.__version__,
        "xgboost": _xgb.__version__, "scikit_learn": _sk.__version__, "numpy": _np.__version__,
    }
    with tempfile.TemporaryDirectory() as td:
        fp = os.path.join(td, "versions.json")
        with open(fp, "w", encoding="utf-8") as f: json.dump(versions, f, indent=2)
        mlflow.log_artifact(fp)  # raíz del run


def _save_matplotlib(fig, name, artifact_path="plots"):
    with tempfile.TemporaryDirectory() as td:
        out = os.path.join(td, name)
        fig.savefig(out, bbox_inches="tight")
        mlflow.log_artifact(out, artifact_path=artifact_path)
    plt.close(fig)

In [ ]:
def optimize_model(X_train, y_train, X_valid, y_valid, experiment_name=None, n_trials=20, random_state=42):
    print("[INFO] Iniciando optimize_model()")
    avg = "binary" if len(np.unique(y_valid)) == 2 else "macro"
    print(f"[INFO] f1 average = {avg}")

    if experiment_name is None:
        experiment_name = f"XGB_Optuna_{datetime.now().strftime('%Y%m%d_%H%M%S')}"
    print(f"[INFO] Usando experimento: {experiment_name}")

    exp = mlflow.get_experiment_by_name(experiment_name)
    experiment_id = exp.experiment_id if exp else mlflow.create_experiment(experiment_name)
    mlflow.set_experiment(experiment_name)

    def objective(trial: optuna.Trial) -> float:
        params = {
            "n_estimators": trial.suggest_int("n_estimators", 100, 600),
            "learning_rate": trial.suggest_float("learning_rate", 1e-3, 0.3, log=True),
            "max_depth": trial.suggest_int("max_depth", 3, 10),
            "subsample": trial.suggest_float("subsample", 0.6, 1.0),
            "colsample_bytree": trial.suggest_float("colsample_bytree", 0.6, 1.0),
            "random_state": random_state,
            "n_jobs": -1,
            "objective": "binary:logistic" if avg == "binary" else "multi:softprob",
            "tree_method": "hist",
            "eval_metric": "logloss",
        }
        run_name = f"XGBoost con lr {params['learning_rate']:.3f}"
        print(f"[TRIAL {trial.number}] {run_name}")

        with mlflow.start_run(run_name=run_name, experiment_id=experiment_id):
            _log_versions()  # útil si el run falla en tu infra
            mlflow.log_params(params)


            early_stop_callback = EarlyStopping(
                rounds=30,  # Same as your original early_stopping_rounds value
                min_delta=1e-5,
                save_best=True,
                # Use maximize=False because 'logloss' (your eval_metric) is minimized
                maximize=False 
            )

            model = XGBClassifier(**params)
            model.set_params(callbacks=[XGBClassifier.callback.EarlyStopping(rounds=10)])
            model.fit(
                X_train, 
                y_train, 
                eval_set=[(X_valid, y_valid)], 
                verbose=False, 
            )

            if avg == "binary":
                y_prob = model.predict_proba(X_valid)[:, 1]
                y_pred = (y_prob >= 0.5).astype(int)
            else:
                y_pred = np.argmax(model.predict_proba(X_valid), axis=1)

            f1 = f1_score(y_valid, y_pred, average=avg)
            print(f"[TRIAL {trial.number}] valid_f1={f1:.5f}")
            mlflow.log_metric("valid_f1", f1)

            # registrar el modelo del trial
            mlflow.sklearn.log_model(model, artifact_path="model")
            return f1

    study = optuna.create_study(direction="maximize")
    study.optimize(objective, n_trials=n_trials, show_progress_bar=False)

    print(f"[INFO] Mejores params: {study.best_params} | best f1={study.best_value:.5f}")

    # Gráficos de Optuna -> /plots
    fig1 = plot_optimization_history(study); _save_matplotlib(fig1.figure, "optuna_optimization_history.png")
    fig2 = plot_param_importances(study);   _save_matplotlib(fig2.figure, "optuna_param_importances.png")

    # Cargar mejor modelo con la función proporcionada (sin modificar)
    print("[INFO] Cargando mejor modelo con get_best_model()…")
    best_model = get_best_model(experiment_id)

    # Guardar config final y feature importance -> /plots
    cfg = getattr(best_model, "get_xgb_params", None)
    best_cfg = cfg() if cfg else best_model.get_params()
    with tempfile.TemporaryDirectory() as td:
        cfgp = os.path.join(td, "final_model_config.json")
        with open(cfgp, "w", encoding="utf-8") as f: json.dump(best_cfg, f, indent=2, default=str)
        mlflow.log_artifact(cfgp, artifact_path="plots")

    try:
        importances = getattr(best_model, "feature_importances_", None)
        if importances is not None:
            order = np.argsort(importances)[::-1][:20]
            fig, ax = plt.subplots(figsize=(8, 5))
            ax.bar(range(len(order)), importances[order])
            ax.set_xticks(range(len(order)))
            ax.set_xticklabels([f"f{i}" for i in order], rotation=45, ha="right")
            ax.set_title("XGBoost - Importancia de variables")
            _save_matplotlib(fig, "final_feature_importances.png", artifact_path="plots")
    except Exception as e:
        print(f"[WARN] No se pudo graficar importancias: {e}")

    # Serializar mejor modelo en /models
    with mlflow.start_run(run_name="Exportar mejor modelo", experiment_id=experiment_id):
        with tempfile.TemporaryDirectory() as td:
            mp = os.path.join(td, "best_model.pkl")
            with open(mp, "wb") as f: pickle.dump(best_model, f)
            mlflow.log_artifact(mp, artifact_path="models")
        print("[INFO] Modelo serializado a /models/best_model.pkl")

    print("[OK] optimize_model terminado.")
    return experiment_id, study

# **2. FastAPI (2.0 puntos)**

<div align="center">
  <img src="https://media3.giphy.com/media/YQitE4YNQNahy/giphy-downsized-large.gif" width="500">
</div>

Con el modelo ya entrenado, la idea de esta sección es generar una API REST a la cual se le pueda hacer *requests* para así interactuar con su modelo. En particular, se le pide:

- Guardar el código de esta sección en el archivo `main.py`. Note que ejecutar `python main.py` debería levantar el servidor en el puerto por defecto.
- Defina `GET` con ruta tipo *home* que describa brevemente su modelo, el problema que intenta resolver, su entrada y salida.
- Defina un `POST` a la ruta `/potabilidad/` donde utilice su mejor optimizado para predecir si una medición de agua es o no potable. Por ejemplo, una llamada de esta ruta con un *body*:

```json
{
   "ph":10.316400384553162,
   "Hardness":217.2668424334475,
   "Solids":10676.508475429378,
   "Chloramines":3.445514571005745,
   "Sulfate":397.7549459751925,
   "Conductivity":492.20647361771086,
   "Organic_carbon":12.812732207582542,
   "Trihalomethanes":72.28192021570328,
   "Turbidity":3.4073494284238364
}
```

Su servidor debería retornar una respuesta HTML con código 200 con:


```json
{
  "potabilidad": 0 # respuesta puede variar según el clasificador que entrenen
}
```

**`HINT:` Recuerde que puede utilizar [http://localhost:8000/docs](http://localhost:8000/docs) para hacer un `POST`.**

# **3. Docker (2 puntos)**

<div align="center">
  <img src="https://miro.medium.com/v2/resize:fit:1400/1*9rafh2W0rbRJIKJzqYc8yA.gif" width="500">
</div>

Tras el éxito de su aplicación web para generar la salida, Smapina le solicita que genere un contenedor para poder ejecutarla en cualquier computador de la empresa de agua potable.

## **3.1 Creación de Container (1 punto)**

Cree un Dockerfile que use una imagen base de Python, copie los archivos del proyecto e instale las dependencias desde un `requirements.txt`. Con esto, construya y ejecute el contenedor Docker para la API configurada anteriormente. Entregue el código fuente (incluyendo `main.py`, `requirements.txt`, y `Dockerfile`) y la imagen Docker de la aplicación. Para la dockerización, asegúrese de cumplir con los siguientes puntos:

1. **Generar un archivo `.dockerignore`** que ignore carpetas y archivos innecesarios dentro del contenedor.
2. **Configurar un volumen** que permita la persistencia de los datos en una ruta local del computador.
3. **Exponer el puerto** para acceder a la ruta de la API sin tener que entrar al contenedor directamente.
4. **Incluir imágenes en el notebook** que muestren la ejecución del contenedor y los resultados obtenidos.
5. **Revisar y comentar los recursos utilizados por el contenedor**. Analice si los contenedores son livianos en términos de recursos.

## **3.2 Preguntas de Smapina (1 punto)**
Tras haber experimentado con Docker, Smapina desea profundizar más en el tema y decide realizarle las siguientes consultas:

- ¿Cómo se diferencia Docker de una máquina virtual (VM)?
- ¿Cuál es la diferencia entre usar Docker y ejecutar la aplicación directamente en el sistema local?
- ¿Cómo asegura Docker la consistencia entre diferentes entornos de desarrollo y producción?
- ¿Cómo se gestionan los volúmenes en Docker para la persistencia de datos?
- ¿Qué son Dockerfile y docker-compose.yml, y cuál es su propósito?

Respuestas

<div align="center">
  <img src="app/imágenes/Screenshot 2025-10-21 at 21.31.13.png" width="500">
</div>

Explicación: En esta captura se observa el proceso de construcción de la imagen Docker mediante el comando
docker build -t miapp ..
Docker utiliza el archivo Dockerfile para crear una imagen basada en python:3.11-slim, copia los archivos del proyecto, instala las dependencias desde requirements.txt y prepara el entorno de ejecución.
La salida muestra cada capa creada y confirma que la imagen miapp:latest fue generada correctamente, cumpliendo con el primer paso del laboratorio.

<div align="center">
  <img src="app/imágenes/Screenshot 2025-10-21 at 21.31.46.png" width="500">
</div>

Explicación: En esta imagen se muestra la ejecución del contenedor con el comando
docker run --rm -p 8000:80 -v "$(pwd)/data:/data" miapp.
La aplicación inicia dentro del contenedor, carga el modelo de ML desde MLflow y muestra los logs de inicialización.
Finalmente, se visualiza el mensaje
Uvicorn running on http://0.0.0.0:80, indicando que el servidor FastAPI está activo dentro del contenedor y accesible desde el host mediante la dirección http://localhost:8000.


<div align="center">
  <img src="app/imágenes/Screenshot 2025-10-21 at 21.32.14.png" width="500">
</div>

Explicación: Esta captura corresponde a la interfaz interactiva Swagger UI generada automáticamente por FastAPI, accesible en
http://localhost:8000/docs.
En ella se observan los endpoints disponibles (/home y /potabilidad/), junto con los esquemas de entrada (WaterSample) y salida.
Esta vista permite probar de forma gráfica la API desde el navegador, demostrando que el contenedor expone correctamente la aplicación y el puerto configurado.


<div align="center">
  <img src="app/imágenes/Screenshot 2025-10-21 at 21.33.11.png" width="500">
</div>

Explicación: En esta figura se muestra una prueba del endpoint /potabilidad/ utilizando el método POST.
Se envía un conjunto de parámetros físico-químicos del agua en formato JSON, y la API responde con la predicción de potabilidad (0).
Esta evidencia confirma el correcto funcionamiento del modelo dentro del contenedor y la comunicación entre el servidor FastAPI y el motor de predicción cargado desde MLflow.


Luego se utiliza el comando docker stats, el cual muestra los recursos utilizados por el contenedor en tiempo real.
En este caso, el contenedor consume aproximadamente 148 MiB de memoria (3.7 %) y menos del 1 % de CPU, lo que indica que la aplicación es liviana y eficiente en ejecución.
El tamaño de la imagen (~1.44 GB) se debe principalmente a dependencias de machine learning como scikit-learn y xgboost, lo cual es esperable para este tipo de proyectos.


---


3.2 Preguntas de Smapina

1.  Docker no crea un sistema operativo completo como una VM, sino que usa el mismo sistema del host.
Por eso arranca mucho más rápido y consume menos recursos. Las VMs son más pesadas porque simulan todo un sistema operativo aparte.


2. Al usar Docker, la app corre dentro de un contenedor aislado, con sus propias dependencias y versión de Python, sin afectar al sistema.
Si se ejecuta localmente, se pueden tener conflictos con librerías o versiones distintas.



3. Se puede asegurar porque se utiliza la misma imagen en ambos contextos.
El contenedor se comporta igual en cualquier máquina, ya que dentro tiene el mismo sistema, librerías y configuración.



4. Los volúmenes permiten guardar datos fuera del contenedor, en una carpeta del computador.
Así, aunque se borre o reinicie el contenedor, los datos quedan guardados.


5. 
* El **Dockerfile** define paso a paso cómo crear la imagen (qué base usar, qué copiar, qué instalar y cómo ejecutar).
* El **docker-compose.yml** sirve para levantar varios contenedores juntos (por ejemplo, backend + base de datos) con un solo comando.




# Conclusión

Éxito!
<div align="center">
  <img src="https://i.pinimg.com/originals/55/f5/fd/55f5fdc9455989f8caf7fca7f93bd96a.gif" width="500">
</div>